In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
         0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
         1, 0, 0, 1]),
 tensor([  77,  486,  435,  425,  169,   95,  170,  517,  253,   56, 1188,  220,
          198,   99,  723, 1288,  371,  387,  415,  208,  187,  364,  646,   95,
          134,  224,  184,  313,  260,  121,   98,  352,  290,   77,  296,  188,
          166,  181,  205,  176,  144,  220,  385,  257,  236,  253,  652,  193,
          288,  347,  356,  411]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-6
bidirectional=True
dropout_e=0.1 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [43]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out=n_out
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden*2,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        #import pdb
        #pdb.set_trace()
        #embs=self.dropout_enc(embs)
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        #import pdb
        #pdb.set_trace()
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,_=pad_packed_sequence(lstm_out,batch_first=True)
        #lstm_out=self.dropout_op(lstm_out)      
        
        # Wrap the hidden state in a new tensor without the gradients
        #self.hidden=(Variable(newhidden[0].data,requires_grad=False).to(self.device),Variable(newhidden[1].data,requires_grad=False).to(self.device))
        
            # lstm_out has the values that are padded
        #out=self.fc (lstm_out[:,-1,:])
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        
        preds=self.fc(hidden)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [44]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [45]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [46]:
device="cpu"

## Load pretrained model weights

In [47]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [48]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [49]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.1, 0.5, 0.5, 1)

In [50]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [51]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [52]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 30,402,401 trainable parameters


In [53]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.1)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=800, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (criterion): BCEWithLogitsLoss()
)

In [54]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0057, -0.0218, -0.0379, -0.0263, -0.0034, -0.0097, -0.0489, -0.0381,
         -0.0459, -0.0648, -0.0513, -0.0650, -0.0613, -0.0114, -0.0384,  0.0117,
         -0.0634, -0.0441, -0.0327, -0.0516, -0.0436, -0.0385, -0.0293, -0.0189,
          0.0221, -0.0306, -0.0268, -0.0068, -0.0590, -0.0556, -0.0035, -0.0688,
         -0.0113, -0.0524, -0.0454, -0.0030, -0.0562, -0.0450, -0.0335, -0.0689,
          0.0234, -0.0523,  0.0024, -0.0027, -0.0411, -0.0300, -0.0654, -0.0126,
         -0.0469, -0.0004, -0.0677, -0.0421], grad_fn=<ViewBackward>),
 tensor(0.6943, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [55]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [56]:
preds.view(-1)

tensor([-2.1524e-02,  2.7905e-02, -5.0202e-02, -5.0593e-02, -1.5948e-02,
        -4.4161e-02, -2.9383e-02, -9.8798e-03, -3.6411e-02,  1.8726e-03,
        -6.3698e-02, -4.1794e-02, -3.3014e-02, -4.9287e-02,  1.1356e-02,
         3.2799e-02, -3.4925e-02, -6.2835e-02, -1.3888e-02, -3.1053e-02,
        -3.7084e-02, -2.3132e-02, -2.6684e-02, -4.1958e-02, -6.4447e-02,
        -2.1188e-02,  5.5239e-05, -3.7687e-02, -4.8288e-02, -5.3419e-02,
        -2.9108e-02, -6.7970e-02, -2.1826e-02, -2.6133e-02, -3.6825e-02,
        -7.4513e-03, -3.6871e-02, -5.2097e-02, -9.4595e-03, -3.4023e-02,
        -2.6630e-02, -3.3766e-02, -3.6976e-02, -3.4707e-02, -6.4169e-02,
        -4.2531e-02, -7.4793e-02, -7.9659e-02, -3.3324e-02, -2.6354e-02,
        -2.6314e-02, -3.7594e-02], grad_fn=<ViewBackward>)

In [57]:
yb

tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 1])

In [58]:
preds.size()

torch.Size([52])

In [59]:
yb.size()

torch.Size([52])

In [60]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5769)

In [61]:
from sklearn.metrics import roc_auc_score

In [62]:
roc_auc_score(yb,preds)

0.45333333333333337

In [63]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [64]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [65]:
device="cuda:1"

In [66]:
n_layers

2

In [67]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [68]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [ ]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o
learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

In [69]:
len(dltrain),len(dlvalid)

(481, 481)

In [70]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [71]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [72]:
model_sentiment.freeze_embedding()

In [73]:
model_sentiment.encoder.weight.requires_grad

False

In [74]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.7066369485855103  0.5430769413709641
Batch:100 0.684576485157013  0.5759615570306778
Batch:150 0.6425048341353734  0.6267948929468791
Batch:200 0.6065327759087086  0.6668269464373588
Batch:250 0.5629986177682876  0.7026154100894928
Batch:300 0.5198106682797273  0.7329487460851669
Batch:350 0.49186822848660605  0.7530220081124986
Batch:400 0.4667872473783791  0.7699519543349743
Batch:450 0.44730751251180967  0.7832478948434194
Epoch:0 Loss:0.4355782112900532
Batch:50 0.23108960017561914  0.9107692682743073
Batch:100 0.22891017936170102  0.9103846526145936
Batch:150 0.22456104099750518  0.9105128578344981
Batch:200 0.22391567029058934  0.9104808065295219
Batch:250 0.22508717516064644  0.9097692675590515
Batch:300 0.22787363258500895  0.9082051652669907
Batch:350 0.23063194313219615  0.9078022347177778
Batch:400 0.2317401228286326  0.907403883934021
Batch:450 0.2332555033100976  0.9073504650592804
Epoch:0 Loss:0.4355782112900532 Accuracy:0.7905365745391766 Loss:0.23505537826838

In [77]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.15643781267106532  0.9392308044433594
Batch:100 0.15170912496745587  0.940576959848404
Batch:150 0.14887252272417148  0.9437179843584697
Batch:200 0.15302244954742492  0.942692342698574
Batch:250 0.15552149205654858  0.9412308051586151
Batch:300 0.1550097617569069  0.9417308056354523
Batch:350 0.15466980420585189  0.9423077288695744
Batch:400 0.1547545985551551  0.9422596521675587
Batch:450 0.1535891719038288  0.9433333696259393
Epoch:0 Loss:0.15388753840200248
Batch:50 0.19780825667083263  0.9261538851261139
Batch:100 0.19303739733994008  0.9257692694664001
Batch:150 0.18850326873362064  0.9271795252958933
Batch:200 0.1865695379115641  0.9279808077216148
Batch:250 0.18671285258233547  0.9276154234409332
Batch:300 0.18851893016447624  0.9266667052110036
Batch:350 0.19104087996695723  0.925494544335774
Batch:400 0.1905733852647245  0.9252885000407696
Batch:450 0.19199975997209548  0.9242308076222737
Epoch:0 Loss:0.15388753840200248 Accuracy:0.9432992525011487 Loss:0.191882620

In [ ]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_nounfreeze_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_nounfreeze_state_dict')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_nounfreeze')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_nounfreeze')
torch.save (learner,f'{PATH}/inter/sentiment_learner_nounfreeze')

In [79]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.13206933081150055  0.9523077249526978
Batch:100 0.13517245573922992  0.9523077273368835
Batch:150 0.13895772594958544  0.9517949072519938
Batch:200 0.13599954378791154  0.9526923432946205
Batch:250 0.13511675421893596  0.9520000364780427
Batch:300 0.13753904967258374  0.9511538829406102
Batch:350 0.13729520365595818  0.951208826984678
Batch:400 0.13575063530821352  0.95125003606081
Batch:450 0.13639881011098623  0.9507265318764581
Epoch:0 Loss:0.13726736557536337
Batch:50 0.20821160793304444  0.9246154248714447
Batch:100 0.20299150850623845  0.9255769610404968
Batch:150 0.19593957920869193  0.9280769618352255
Batch:200 0.19357149995863437  0.9278846541047097
Batch:250 0.19429796694219112  0.9273846545219422
Batch:300 0.19594232687105737  0.9262820903460185
Batch:350 0.19837450975818294  0.925164874451501
Batch:400 0.1978247296344489  0.9250481155514717
Batch:450 0.19936219852003786  0.9242735426955753
Epoch:0 Loss:0.13726736557536337 Accuracy:0.9506797100303079 Loss:0.199030

In [81]:
model_sentiment=torch.load(f'{PATH}/inter/sentiment_model_nounfreeze')
optimizer=torch.load(f'{PATH}/inter/sentiment_optimizer_nounfreeze')
learner=torch.load(f'{PATH}/inter/sentiment_learner_nounfreeze')

In [82]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-4

In [83]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.14123125590384006  0.9515384960174561
Batch:100 0.13720126066356897  0.9519231110811234
Batch:150 0.14175206201771895  0.9503846498330434
Batch:200 0.13787070576101543  0.950673111975193
Batch:250 0.13643595884740353  0.9506923418045043
Batch:300 0.13587378985558948  0.9508974699179331
Batch:350 0.1364343300940735  0.9503846493789128
Batch:400 0.1369226293871179  0.9502884954214096
Batch:450 0.137931289759775  0.9493162737952339
Epoch:0 Loss:0.13690404950596563
Batch:50 0.20945526883006096  0.9242308080196381
Batch:100 0.20437140921130775  0.92538465321064
Batch:150 0.19885895226150752  0.9276923461755117
Batch:200 0.19594240334816276  0.9286538848280906
Batch:250 0.196985928542912  0.9280769622325897
Batch:300 0.19861771863574784  0.9266667060057322
Batch:350 0.20131355455411332  0.9254395999227252
Batch:400 0.20086622369941323  0.925576962530613
Batch:450 0.20266997471037837  0.9247863643699222
Epoch:0 Loss:0.13690404950596563 Accuracy:0.9499320670373722 Loss:0.20239591946

In [78]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_nounfreeze_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_nounfreeze_state_dict')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_nounfreeze')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_nounfreeze')
torch.save (learner,f'{PATH}/inter/sentiment_learner_nounfreeze')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [84]:
model_sentiment.unfreeze_embedding()

In [85]:
model_sentiment.encoder.weight.requires_grad

True

In [86]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.10892339136451483  0.9611538779735566
Batch:100 0.11346936272457242  0.9590384936332703
Batch:150 0.11927967343479395  0.9567949052651723
Batch:200 0.12147471337579191  0.9556731104850769
Batch:250 0.11969630967825651  0.9563846490383148
Batch:300 0.11972544053569437  0.9562820849816004
Batch:350 0.1217567585568343  0.955329703773771
Batch:400 0.1228498473810032  0.9552404180169105
Batch:450 0.12125151558054818  0.9561111439598932
Epoch:0 Loss:0.12041834579575458
Batch:50 0.21568004801869392  0.9215384960174561
Batch:100 0.20902543500065804  0.9248077273368835
Batch:150 0.20488682145873705  0.9260256787141165
Batch:200 0.20381532108411193  0.9253846538066864
Batch:250 0.20431856028735637  0.9250000388622284
Batch:300 0.20576626198987166  0.9237820907433828
Batch:350 0.20771939370248999  0.9228022374425615
Batch:400 0.20725659501738847  0.9228846544027328
Batch:450 0.20813429785271487  0.9223932010597653
Epoch:0 Loss:0.12041834579575458 Accuracy:0.9564089564424543 Loss:0.2072

In [87]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [88]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.0991195897012949  0.968846184015274
Batch:100 0.09707770511507988  0.9696154141426087
Batch:150 0.09497584269692501  0.9682051571210225
Batch:200 0.09389502447796985  0.9688461831212044
Batch:250 0.09578236123733222  0.9685384905338288
Batch:300 0.09489997792833794  0.9682051567236583
Batch:350 0.09468853758781084  0.9682967323916299
Batch:400 0.0958864542923402  0.9677884908020497
Batch:450 0.0963372344058007  0.968076952430937
Epoch:0 Loss:0.097667737183376
Batch:50 0.22544684886932373  0.9230769610404969
Batch:100 0.21884346472099422  0.9242308074235916
Batch:150 0.21248358910282453  0.9269231140613556
Batch:200 0.20938478557392956  0.9275961908698082
Batch:250 0.2092197651565075  0.9276154222488403
Batch:300 0.21024449918419122  0.9268590128421783
Batch:350 0.2124457100672381  0.9256593792779105
Batch:400 0.21124606043100358  0.9261538840830326
Batch:450 0.2128145018219948  0.9253846534093221
Epoch:0 Loss:0.097667737183376 Accuracy:0.9676435608378071 Loss:0.2124421164749

In [89]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.09524211082607507  0.9680769574642182
Batch:100 0.093828199878335  0.9686538773775101
Batch:150 0.0919878846531113  0.9687179811795552
Batch:200 0.09395078460685909  0.9689423388242722
Batch:250 0.09284830520302058  0.9690769546031952
Batch:300 0.09200737948218982  0.9695513131221135
Batch:350 0.09359771828566278  0.9695055254868099
Batch:400 0.09153297192882746  0.9706731064617634
Batch:450 0.09010110232565138  0.971068405840132
Epoch:0 Loss:0.09070706779714248
Batch:50 0.23427742831408976  0.9230769610404969
Batch:100 0.22739623751491309  0.9246154230833054
Batch:150 0.22029945831745862  0.9264102943738302
Batch:200 0.21663093800656497  0.9272115764021873
Batch:250 0.216403607301414  0.927153883934021
Batch:300 0.2173488519154489  0.9263461919625601
Batch:350 0.21949118124055012  0.9253846536363874
Batch:400 0.2178897394472733  0.92600965321064
Batch:450 0.2196268634787864  0.9252137135134803
Epoch:0 Loss:0.09070706779714248 Accuracy:0.9704582101342089 Loss:0.2193625805214

In [179]:
len(dltrain)

433